# DEPENDENCE

In [1]:
import sys, os
import datetime, time
import traceback, logging
import pandas as pd
import tkinter
import tkinter.messagebox
import json

BASE_DIR = r"C:\Users\leo.zhangzs\AppData\Roaming\Code\User\VScode\py3\UAT_Test"
sys.path.append(BASE_DIR)

from pytz import utc
from string import Template
from multiprocessing import Process
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.jobstores.memory import MemoryJobStore
from apscheduler.events import EVENT_JOB_EXECUTED, EVENT_JOB_ERROR, EVENT_JOB_MISSED, EVENT_SCHEDULER_STARTED, EVENT_SCHEDULER_SHUTDOWN
from apscheduler.executors.pool import ThreadPoolExecutor, ProcessPoolExecutor
from apscheduler.triggers.cron import CronTrigger

from core.configadmin import ConfigAdmin
from core.color_me import ColorMe
from core import log_ctrl
from core.outlook import send_email
from core.cron import Cron
from core.monitor_check import MonitorCheck

jobstores = {
    'default':MemoryJobStore()
}
executors = {
    'default':ThreadPoolExecutor(20),
    'processpool':ProcessPoolExecutor(10)
}
job_defaults = {
    'coalesce':False,
    'max_instances':1,
    'misfire_grace_time':10
}

config_path = os.path.join(BASE_DIR, 'conf')
config_name = 'setting.ini'
configs = ConfigAdmin(config_path, config_name)
Job_List = os.path.join(BASE_DIR, 'db', 'Sche', 'MonitorList.xlsx')
logs = log_ctrl.de8ug_log(logger_name='Main_DWH', log_file=os.path.join(BASE_DIR, 'log', f'{time.strftime("%Y%m",time.localtime())}-run.log'), level = logging.INFO)
# , level = logging.INFO)
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(filename)s[line:%(lineno)d] %(levelname)s %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    filename=os.path.join(BASE_DIR, 'log', f'{time.strftime("%Y%m",time.localtime())}-scheduler.log'),
                    filemode='a')
                    
scheduler = BlockingScheduler(jobstores=jobstores, executors=executors, job_defaults=job_defaults, timezone=utc)
scheduler._logger = logging
# panda option
pd.set_option('max_colwidth',200)

# DESIGN

## Class

In [2]:
def main(Jname, **kwargs):
    print(datetime.datetime.strftime(datetime.datetime.now(), "%Y/%m/%d %X"), 'Start-', Jname, os.getpid())
    # print(kwargs)
    try:
        Rest = MonitorCheck(kwargs)
        if Rest["Status"] == "True":
            if int(kwargs['SLEEP_TIME'])>0:
                scheduler.pause_job(Jname)
                time.sleep(int(kwargs['SLEEP_TIME']))
                logs.info(Jname, f'Wait {kwargs["SLEEP_TIME"]} sec...')
                scheduler.resume_job(Jname)
        else:
            ErrorAlert(Rest["msg"])
            scheduler.pause_job(Jname)
    except Exception as e:
        print(e)
        os._exit(0)

def Listener(event):
    if event.exception:
        logging.error(event.exception)
        print ('JOB_ERROR')
    elif EVENT_JOB_EXECUTED:
        pass
        # print (EVENT_JOB_EXECUTED, 'JOB_EXECUTED')
    elif EVENT_JOB_MISSED:
        print (EVENT_JOB_MISSED, 'JOB_MISSED')
    else:
        pass
    
def ErrorAlert(msg):
    TempError = os.path.join(BASE_DIR, 'conf', 'CSS', configs.read_config('CSS_Style','errortemplate'))
    sent_to = 'Leo.ZhangZS@homecredit.cn'
    subject = 'Program Error'
    ProcessName = 'main_monitor_alert.py'
    EffectLevel = 'High'
    ContactNotice = 'Leo.ZhangZS'
    RescueOperation = 'Eorror Info'
    EorrorTime = datetime.datetime.strftime(datetime.datetime.now(), "%Y/%m/%d %X")
    with open(TempError, 'r') as f:
        t1 = Template(f.read())
        content_html = t1.substitute(ProcName=ProcessName, EffectLevel=EffectLevel, ContactNotice=ContactNotice, ResOpt=RescueOperation, EorrTime=EorrorTime, EorrInfo=msg)
    send_email.outlook(sent_to, subject, content_html, None, None)

## ClassTest

## Main

In [ ]:
if __name__ == "__main__":
    """
    无状态监控任务调度:
    1.读取Excel中的任务列表
    2.执行SQL脚本
    3.检查是否达成报警条件
    4.发送执行结果通知
    """
    try:
        scheduler.add_listener(Listener, EVENT_JOB_EXECUTED | EVENT_JOB_ERROR | EVENT_JOB_MISSED)
        print(datetime.datetime.strftime(datetime.datetime.now(), "%Y/%m/%d %X"), 'Programe Start -', os.getpid())
        df = pd.read_excel(Job_List, 'Sheet1', convert_float=False)
        Job_List = df[(df['STATUS']=="A") & (df['SCRIPTS'].isnull() == False) & df.SCRIPTS.apply(lambda x: os.path.exists(os.path.join(BASE_DIR, 'db', 'sql', str(x))) == True)]
        for i in Job_List.index:
            f1 = Job_List.loc[[i]]
            js = json.loads(f1.to_json(orient='records'))[0]
            JobNam = js['NAME']
            Trriger = js['TRIGGER_TYPE']
            Time = js['TRIGGER_VALUE']
            if Trriger == 'CRON':
                year = str(Time).split(' ')[5]
                if year in ['0','1','2','3','4','5','6']:
                    week = year
                    year = None
                else:
                    week = None
                month = str(Time).split(' ')[4]
                day = str(Time).split(' ')[3]
                hour = str(Time).split(' ')[2]
                minute = str(Time).split(' ')[1]
                second = str(Time).split(' ')[0]
                # print(week, year, month, day, hour, minute, second)
                scheduler.add_job(func=main, args=[JobNam], kwargs=js, id=JobNam, trigger='cron', year=year, month=month, day=day, hour=hour, minute=minute, second=second, timezone='Asia/Shanghai', replace_existing=True)
            elif Trriger == 'DATE':
                scheduler.add_job(func=main, args=[JobNam], kwargs=js, id=JobNam, trigger='date', run_date=datetime.datetime.utcfromtimestamp(Time/1000))
            elif Trriger == 'INTERVAL':
                sec = int(Time)
                scheduler.add_job(func=main, args=[JobNam], kwargs=js, id=JobNam, trigger='interval', seconds=sec, replace_existing=True)
        print('Press Ctrl+{0} to exit'.format('Break' if os.name == 'nt' else 'C'))
        scheduler.start()
    except (KeyboardInterrupt, SystemExit):
        scheduler.shutdown()
        print(datetime.datetime.strftime(datetime.datetime.now(), "%Y/%m/%d %X"), "Exit Programe")
    except Exception as e:
        logs.critical(traceback.format_exc())
        ErrorAlert(e)
        tkinter.messagebox.showerror("ERROR", e)
        os._exit(0)

## MainTest

In [3]:
df = pd.read_excel(Job_List, 'Sheet1', convert_float=False)
Job_List = df[(df['STATUS']=="A") & (df['SCRIPTS'].isnull() == False) & df.SCRIPTS.apply(lambda x: os.path.exists(os.path.join(BASE_DIR, 'db', 'sql', str(x))) == True)]
Job_List

,NAME,TRIGGER_TYPE,TRIGGER_VALUE,SLEEP_TIME,STATUS,DB,SCRIPTS,SCRIPTS_TYPE,INPUT,CONDITION,NOTIFY,SENTER,TEMPLATE,OUTPUT,PRINCIPAL
8,Canceled Campaign D-1,CRON,0 22 11 * * *,0.0,A,CRM,Canceled_Campaign_Yesterday.sql,EXECUTE,NaN,NaN,Y,Leo.ZhangZS@homecredit.cn,Canceled Campaign D-1.html,NaN,NaN


In [4]:
f1 = Job_List.loc[[8]]
js = json.loads(f1.to_json(orient='records'))[0]

In [5]:
js

{'NAME': 'Canceled Campaign D-1',
 'TRIGGER_TYPE': 'CRON',
 'TRIGGER_VALUE': '0 22 11 * * *',
 'SLEEP_TIME': 0.0,
 'STATUS': 'A',
 'DB': 'CRM',
 'SCRIPTS': 'Canceled_Campaign_Yesterday.sql',
 'SCRIPTS_TYPE': 'EXECUTE',
 'INPUT': None,
 'CONDITION': None,
 'NOTIFY': 'Y',
 'SENTER': 'Leo.ZhangZS@homecredit.cn',
 'TEMPLATE': 'Canceled Campaign D-1.html',
 'OUTPUT': None,
 'PRINCIPAL': None}

In [6]:
res = MonitorCheck(js).Check()
print(res['Status'],'\r\n', res['msg'])
# # datetime.datetime.strftime(datetime.datetime.utcfromtimestamp(js['TRIGGER_VALUE']/1000), '%Y-%m-%d %X')

2020-08-21 11:26:53,897 sqlplus.py [line:57] sqlplus INFO 执行成功
2020-08-21 11:26:53,924 monitor_check.py [line:151] Main_DWH ERROR Traceback (most recent call last):
  File "C:\Users\leo.zhangzs\AppData\Roaming\Code\User\VScode\py3\UAT_Test\core\monitor_check.py", line 120, in Check
    Htable = df.to_html(columns=df.columns.tolist(), index=False, border=1, classes='fl-table')
  File "D:\Program\miniconda\envs\py36_32\lib\site-packages\pandas\core\frame.py", line 1584, in to_html
    formatter.to_html(classes=classes, notebook=notebook, border=border)
  File "D:\Program\miniconda\envs\py36_32\lib\site-packages\pandas\io\formats\format.py", line 721, in to_html
    html_renderer.write_result(self.buf)
  File "D:\Program\miniconda\envs\py36_32\lib\site-packages\pandas\io\formats\format.py", line 1166, in write_result
    indent = self._write_body(indent)
  File "D:\Program\miniconda\envs\py36_32\lib\site-packages\pandas\io\formats\format.py", line 1330, in _write_body
    row = [fmt_value

Error 
 Traceback (most recent call last):
  File "C:\Users\leo.zhangzs\AppData\Roaming\Code\User\VScode\py3\UAT_Test\core\monitor_check.py", line 120, in Check
    Htable = df.to_html(columns=df.columns.tolist(), index=False, border=1, classes='fl-table')
  File "D:\Program\miniconda\envs\py36_32\lib\site-packages\pandas\core\frame.py", line 1584, in to_html
    formatter.to_html(classes=classes, notebook=notebook, border=border)
  File "D:\Program\miniconda\envs\py36_32\lib\site-packages\pandas\io\formats\format.py", line 721, in to_html
    html_renderer.write_result(self.buf)
  File "D:\Program\miniconda\envs\py36_32\lib\site-packages\pandas\io\formats\format.py", line 1166, in write_result
    indent = self._write_body(indent)
  File "D:\Program\miniconda\envs\py36_32\lib\site-packages\pandas\io\formats\format.py", line 1330, in _write_body
    row = [fmt_values[j][i] for j in range(len(self.columns))]
  File "D:\Program\miniconda\envs\py36_32\lib\site-packages\pandas\io\formats\f

# TEST

## Model Test

### MonitorCheck

In [7]:
from string import Template
from core.configadmin import ConfigAdmin
from core.sqlplus import SqlPlus
from core.outlook import send_email
from core import log_ctrl

In [9]:
inputJson = js
DB = str(inputJson['DB'])
db_user = configs.read_config("SAS_db", f"{DB.lower()}_user")
db_pswd = configs.read_config("SAS_db", f"{DB.lower()}_password")
db_serv = configs.read_config("SAS_db", f"{DB.lower()}_service")
SCRIPTS_TYPE = inputJson['SCRIPTS_TYPE']
SCRIPTS  = inputJson['SCRIPTS']
SLEEP_TIME = int(inputJson['SLEEP_TIME'])
if inputJson['INPUT'] != None:
    INPUT = json.loads(inputJson['INPUT'])
else:
    INPUT = None
CONDITION = inputJson['CONDITION']
NOTIFY = inputJson['NOTIFY']
SENDER = inputJson['SENTER']
TEMPLATE = os.path.join(BASE_DIR, 'conf', 'CSS', inputJson['TEMPLATE'])
NAME = inputJson['NAME']
PRINCIPAL = inputJson['PRINCIPAL']
if inputJson['OUTPUT'] != None:
    op = inputJson['OUTPUT'].split(' ')
else:
    op = None
STATUS = True
msg = None

In [10]:
with open(os.path.join(BASE_DIR, 'db', 'sql', SCRIPTS), 'r', encoding='utf-8') as f:
    if INPUT != None:
        sql = Template(f.read()).substitute(INPUT)
    else:
        sql = f.read()
res = SqlPlus(db_user, db_pswd, db_serv).fetchall(sql)

2020-08-21 11:31:53,292 sqlplus.py [line:57] sqlplus INFO 执行成功


In [11]:
res['msg']

,CAMPAIGN_NM,CAMPAIGN_CD,COMMUNICATION_NM,SERVICE_CODE,STATUS,STATUS,PROCESSED_DTTM
Daily_Remove_HCP_contract_Confirmation_CALL,Daily_Remove_HCP_contract_Confirmation_CALL,CAMP2021,Remove_HCP_CC_Call,CANCEL_ENTITY,CONSUMED,p,2020-08-20 20:10:19
Daily_Remove_HCP_contract_Confirmation_CALL,Daily_Remove_HCP_contract_Confirmation_CALL,CAMP2021,Remove_HCP_CC_CALL_T+7,CANCEL_ENTITY,CONSUMED,p,2020-08-20 20:14:36
Remove_RTDM_AF_CALL,Remove_RTDM_AF_CALL,CAMP4203,Cancel RTDM call,CANCEL_ENTITY,CONSUMED,p,2020-08-20 20:30:18
Remove_RHCP_CALL,Remove_RHCP_CALL,CAMP2982,Remove_HCP_Mul_CALL,CANCEL_ENTITY,CONSUMED,p,2020-08-20 21:01:13
Remove_Reminding_CALL,Remove_Reminding_CALL,CAMP230,Remove Contract Confirmation Call,CANCEL_ENTITY,CONSUMED,p,2020-08-20 20:25:24
Remove_Call_ORBP_After_Scoring_Intention_New,Remove_Call_ORBP_After_Scoring_Intention_New,CAMP775,cancel_after_score,CANCEL_ENTITY,CONSUMED,p,2020-08-20 18:47:47
Remove_HCP_CALL,Remove_HCP_CALL,CAMP1743,Remove_HCP AF Call,CANCEL_ENTITY,CONSUMED,p,2020-08-20 19:51:21
Remove_HCP_CALL,Remove_HCP_CALL,CAMP1743,Remove_HCP_General_CALL,CANCEL_ENTITY,CONSUMED,p,2020-08-20 19:40:19
Remove_Call_ORBP_MW,Remove_Call_ORBP_MW,CAMP5528,Cancel_Call_S_MW,CANCEL_ENTITY,CONSUMED,p,2020-08-20 20:00:19
Remove_Call_ORBP_After_Scoring_MW,Remove_Call_ORBP_After_Scoring_MW,CAMP5529,Cancel_Call_P_MW,CANCEL_ENTITY,CONSUMED,p,2020-08-20 20:00:58


In [ ]:
df = res['msg']
check = []
for i in df.index:
    locals = (json.loads(df.loc[[i]].to_json(orient='records'))[0])
    res = eval(CONDITION, locals)
    df.loc[i, 'CHECK'] = res
#     if res != None:
#         check.append(res)

In [22]:
df = res['msg']
# df.columns.tolist()
df.to_html(index=False, border=1, classes='fl-table')

'<table border="1" class="dataframe fl-table">\n  <thead>\n    <tr style="text-align: right;">\n      <th>CAMPAIGN_NM</th>\n      <th>CAMPAIGN_CD</th>\n      <th>COMMUNICATION_NM</th>\n      <th>SERVICE_CODE</th>\n      <th>STATUS</th>\n      <th>STATUS</th>\n      <th>PROCESSED_DTTM</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Daily_Remove_HCP_contract_Confirmation_CALL</td>\n      <td>CAMP2021</td>\n      <td>Remove_HCP_CC_Call</td>\n      <td>CANCEL_ENTITY</td>\n      <td>CONSUMED</td>\n      <td>p</td>\n      <td>2020-08-20 20:10:19</td>\n    </tr>\n    <tr>\n      <td>Daily_Remove_HCP_contract_Confirmation_CALL</td>\n      <td>CAMP2021</td>\n      <td>Remove_HCP_CC_CALL_T+7</td>\n      <td>CANCEL_ENTITY</td>\n      <td>CONSUMED</td>\n      <td>p</td>\n      <td>2020-08-20 20:14:36</td>\n    </tr>\n    <tr>\n      <td>Remove_RTDM_AF_CALL</td>\n      <td>CAMP4203</td>\n      <td>Cancel RTDM call</td>\n      <td>CANCEL_ENTITY</td>\n      <td>CONSUMED</td>\n      <td>p<

In [21]:
print(df.to_html(index=False, border=1, classes='fl-table'))

<table border="1" class="dataframe fl-table">
  <thead>
    <tr style="text-align: right;">
      <th>CAMPAIGN_NM</th>
      <th>CAMPAIGN_CD</th>
      <th>COMMUNICATION_NM</th>
      <th>SERVICE_CODE</th>
      <th>STATUS</th>
      <th>STATUS</th>
      <th>PROCESSED_DTTM</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Daily_Remove_HCP_contract_Confirmation_CALL</td>
      <td>CAMP2021</td>
      <td>Remove_HCP_CC_Call</td>
      <td>CANCEL_ENTITY</td>
      <td>CONSUMED</td>
      <td>p</td>
      <td>2020-08-20 20:10:19</td>
    </tr>
    <tr>
      <td>Daily_Remove_HCP_contract_Confirmation_CALL</td>
      <td>CAMP2021</td>
      <td>Remove_HCP_CC_CALL_T+7</td>
      <td>CANCEL_ENTITY</td>
      <td>CONSUMED</td>
      <td>p</td>
      <td>2020-08-20 20:14:36</td>
    </tr>
    <tr>
      <td>Remove_RTDM_AF_CALL</td>
      <td>CAMP4203</td>
      <td>Cancel RTDM call</td>
      <td>CANCEL_ENTITY</td>
      <td>CONSUMED</td>
      <td>p</td>
      <td>2020-08-20 20:30:18</td

# PRO Test

## Import

In [1]:
from os import truncate
import sys
import os
import time
import traceback
import logging
from numpy.core.fromnumeric import trace
import pandas as pd
import tkinter
import tkinter.messagebox
import json
from datetime import datetime

BASE_DIR = r"C:\Users\leo.zhangzs\AppData\Roaming\Code\User\VScode\py3\schedule_monitor"
sys.path.append(BASE_DIR)

from pytz import utc
from string import Template
from multiprocessing import Process
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.jobstores.memory import MemoryJobStore
from apscheduler.events import EVENT_JOB_EXECUTED, EVENT_JOB_ERROR, EVENT_JOB_MISSED
from apscheduler.executors.pool import ThreadPoolExecutor, ProcessPoolExecutor
from apscheduler.triggers.cron import CronTrigger

from core.configadmin import ConfigAdmin
from core.color_me import ColorMe
from core import log_ctrl
from core.outlook import send_email
from core.monitor_check import MonitorCheck
from core.GetPiloty import GetPiloty

In [2]:
jobstores = {'default': MemoryJobStore()}
executors = {
    'default': ThreadPoolExecutor(20),
    'processpool': ProcessPoolExecutor(10)}
job_defaults = {'coalesce': True, 'max_instances': 2, 'misfire_grace_time': 5}

config_path = os.path.join(BASE_DIR, 'conf')
configs = ConfigAdmin(config_path, 'setting.ini')
pconf = ConfigAdmin(config_path, "pilotyConfiguring.ini")

logs = log_ctrl.de8ug_log(
    logger_name='Main_DWH',
    log_file=os.path.join(
        BASE_DIR,
        'log',
        f'{time.strftime("%Y%m",time.localtime())}-run.log'),
    level=logging.INFO)
# , level = logging.INFO)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(filename)s[line:%(lineno)d] %(levelname)s %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    filename=os.path.join(
        BASE_DIR, 'log', 'scheduler',
        f'{time.strftime("%Y%m%d",time.localtime())}-scheduler.log'),
    filemode='a')
scheduler = BackgroundScheduler(jobstores=jobstores,
                                executors=executors,
                                job_defaults=job_defaults,
                                timezone=utc)
scheduler._logger = logging
# panda option
pd.set_option('max_colwidth', 200)

## Main

### Single Test

In [62]:
# class REG(object):
#     def __init__(self , **kwargs):
        
def Regular(**kwargs):
    print(kwargs)
    Jname=kwargs['NAME']
    StartTime = datetime.strftime(datetime.now(), "%m/%d %X")
#     print(StartTime, 'Start-', Jname, os.getpid())
    try:
        Rest = MonitorCheck(kwargs).getResult()
#         EndTime = datetime.strftime(datetime.now(), "%m/%d %X")
        if Rest["Status"] is False:
            print(f'{StartTime} -- {EndTime} {Jname} {ColorMe("Failed").yellow()}: Wait 10 mins.')
#             scheduler.pause_job(Jname)
#             ErrorAlert(Jname, Rest["msg"])
#             time.sleep(600)
#             scheduler.resume_job(Jname)
        elif Rest["Status"] == "Error":
            print(f'{StartTime} -- {EndTime} {Jname} {ColorMe("Error").red()}: Wait 2 hours.')
#             ErrorAlert(Jname, Rest["msg"])
#             scheduler.pause_job(Jname)
#             time.sleep(7200)
#             scheduler.resume_job(Jname)
        else:
            df = Rest['msg']
            schl = json.loads(kwargs['CONDITION'])
            js = json.loads(
                    pconf.read_config(
                        kwargs['NAME'], 'setting').replace("'", '"'))
            if kwargs['SCRIPTS_TYPE'] == 'AGGREGATE':
                if schl['AGG'] == 'SUM':
                    adf = df.groupby(
                        schl['lable'], as_index=False).sum()
                elif schl['AGG'] == 'COUNT':
                    adf = df.groupby(
                        schl['lable'], as_index=False).count()
                else:
                    raise ValueError("Not Valid Aggregate Value.")

                for a in js['Date']:
                    a['labels'] = adf[a['labels']]
                    a['values'] = adf[a['values']]

            figure = GetPiloty(js)
            if kwargs['NOTIFY'] == 'Y':
                op = kwargs['OUTPUT']
                SENTER = kwargs['SENTER']
                TEMPLATE = os.path.join(
                    BASE_DIR, 'conf', 'CSS', kwargs['TEMPLATE'])
                subject = kwargs['NAME']
                Htable = ''
                if op is not None and len(op) >= 1:
                    Htable = df.to_html(
                        columns=op.split(' '),
                        index=False,
                        border=1,
                        classes='fl-table')
                else:
                    Htable = df.to_html(
                        index=False,
                        border=1,
                        classes='fl-table')
                with open(TEMPLATE, 'r', encoding='utf-8') as f:
                    content = Template(
                        f.read()).substitute(
                                rtitle=subject,
                                img=f"{os.path.basename(figure.msg)}",
                                tableframe=Htable
                                ).replace(
                                    'dataframe fl-table', 'fl-table')
                e = send_email.outlook(
                    SENTER, subject, content, None, figure.msg)
                if e is False:
                    raise Exception('Send Email Fail!')
            EndTime = datetime.strftime(datetime.now(), "%m/%d %X")
            print(f"{StartTime} -- {EndTime} {Jname} {ColorMe('Success!').green()}")
    except Exception as e:
        logs.error(traceback.format_exc())
        
#         ErrorAlert(Jname, e)

In [ ]:
def load(func, js, scheduler):
    JobNam = js['NAME']
    Trriger = js['TRIGGER_TYPE']
    Time = js['TRIGGER_VALUE']
    now = datetime.strftime(datetime.now(), "%y/%m/%d %X")
    print(f'Add Job {JobNam}')
    if Trriger == 'CRON':
        month = str(Time).split(' ')[4]
        day = str(Time).split(' ')[3]
        hour = str(Time).split(' ')[2]
        minute = str(Time).split(' ')[1]
        second = str(Time).split(' ')[0]
        year = str(Time).split(' ')[5]
        if len(year) == 4 or year == "*":
            scheduler.add_job(
                func=func,
                args=[JobNam],
                kwargs=js,
                id=JobNam,
                trigger='cron',
                year=year,
                month=month,
                day=day,
                hour=hour,
                minute=minute,
                second=second,
                timezone='Asia/Shanghai',
                replace_existing=True)
        else:
            scheduler.add_job(
                func=func,
                args=[JobNam],
                kwargs=js,
                id=JobNam,
                trigger='cron',
                day_of_week=year,
                month=month,
                day=day,
                hour=hour,
                minute=minute,
                second=second,
                timezone='Asia/Shanghai',
                replace_existing=True)
    elif Trriger == 'DATE':
        scheduler.add_job(
            func=func,
            args=[JobNam],
            kwargs=js,
            id=JobNam,
            trigger='date',
            run_date=Time)
    elif Trriger == 'INTERVAL':
        start_date = now if js['START_DATE'] is None else js['START_DATE']
        sec = int(Time)
        scheduler.add_job(
            func=func,
            args=[JobNam],
            kwargs=js,
            id=JobNam,
            trigger='interval',
            start_date=start_date,
            seconds=sec,
            replace_existing=True)

In [36]:
def start(scheduler):
#     Job_List = os.path.join(BASE_DIR, 'db', 'Sche', 'MonitorList.xlsx')
    Rept_List = os.path.join(BASE_DIR, 'db', 'Sche', 'ReportList.xlsx')
#     df = pd.read_excel(Job_List, 'Sheet1', convert_float=False)
    df1 = pd.read_excel(Rept_List, 'Sheet1', convert_float=False)
#     Job_List = df[
#         (df['STATUS'] == "A") &
#         (df['SCRIPTS'].isnull() == False) &
#         df.SCRIPTS.apply(lambda x: os.path.exists(
#             os.path.join(BASE_DIR, 'db', 'sql', str(x))) == True)]
    Rept_List = df1[
        (df1['STATUS'] == "A") &
        (df1['SCRIPTS'].isnull() == False) &
        df1.SCRIPTS.apply(lambda x: os.path.exists(
            os.path.join(BASE_DIR, 'db', 'sql', str(x))) == True)]
    jlist = []
    dc ={}
#     for i in Job_List.index:
#         f1 = Job_List.loc[[i]]
#         js = json.loads(f1.to_json(orient='records'))[0]
#         jlist.append(js['NAME'])
#         load(main, js, scheduler)

    for i in Rept_List.index:
        f1 = Rept_List.loc[[i]]
        js = json.loads(f1.to_json(orient='records'))[0]
#         jlist.append(js['NAME'])
#         dc[js['NAME']]=js
        jlist.append(js)
#         load(Regular, js, scheduler)
    return jlist

In [44]:
ts = start(scheduler)

In [58]:
ts[0]

{'NAME': 'Campaign_Category_Weekly_Check',
 'TRIGGER_TYPE': 'CRON',
 'TRIGGER_VALUE': '0 27 13 * * 0',
 'START_DATE': None,
 'SLEEP_TIME': 0.0,
 'STATUS': 'A',
 'DB': 'CRM',
 'SCRIPTS': 'Campaign_Category_Weekly_Check.sql',
 'SCRIPTS_TYPE': 'AGGREGATE',
 'INPUT': None,
 'CONDITION': '{"AGG":"SUM","lable":["FLAG_TRIGGER","FLAG_PRODUCT"],"Value":"msg"}',
 'NULLABLE': 'N',
 'NOTIFY': 'Y',
 'SENTER': 'CRMCampaignTeam@homecredit.cn',
 'TEMPLATE': 'Report_Check.html',
 'OUTPUT': None,
 'PRINCIPAL': 'Leo.ZhangZS'}

In [63]:
# rs = 
Regular(**ts[0])

{'NAME': 'Campaign_Category_Weekly_Check', 'TRIGGER_TYPE': 'CRON', 'TRIGGER_VALUE': '0 27 13 * * 0', 'START_DATE': None, 'SLEEP_TIME': 0.0, 'STATUS': 'A', 'DB': 'CRM', 'SCRIPTS': 'Campaign_Category_Weekly_Check.sql', 'SCRIPTS_TYPE': 'AGGREGATE', 'INPUT': None, 'CONDITION': '{"AGG":"SUM","lable":["FLAG_TRIGGER","FLAG_PRODUCT"],"Value":"msg"}', 'NULLABLE': 'N', 'NOTIFY': 'Y', 'SENTER': 'CRMCampaignTeam@homecredit.cn', 'TEMPLATE': 'Report_Check.html', 'OUTPUT': None, 'PRINCIPAL': 'Leo.ZhangZS'}
12/14 17:16:42 -- 12/14 17:17:01 Campaign_Category_Weekly_Check Success!
